In [2]:
!pip install plotly

In [18]:
!pip install nbformat --upgrade


In [17]:
!pip install pandas

In [2]:

#import relevant libraries
import pandas as pd
from datetime import datetime as dt 
from datetime import timedelta
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors

In [3]:
#import the csv file for the data
data = pd.read_csv('/Users/mwefa/OneDrive/Documents/Data_analysis_Portfolio/PYTHON/Online_retail_Data/online_retail.csv')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
data.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [5]:
#drops any blanks, by only considering the customerID column
#inplace=True does not return a new dataframe, it updates 
#by removing missing values we ensure the anaylsis of customer segmentation in done right
data.dropna(subset=['CustomerID'],inplace=True)

In [6]:
# Removing duplicates (Values in all columns are identical)
print("Number of duplicates before cleaning:",data.duplicated().sum())
data = data.drop_duplicates(keep="first")
print("Number of duplicates after cleaning:",data.duplicated().sum())

Number of duplicates before cleaning: 5225
Number of duplicates after cleaning: 0


In [7]:
#convert the date in data to a datetime format
data['InvoiceDate']=pd.to_datetime(data['InvoiceDate'])
data['TotalAmount']=data['Quantity']* data['UnitPrice']

In [8]:
#checks the first 5 rows
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [176]:
#this is a reference date of today, if you are working with a realtime dataset
#reference_date = pd.Timestamp(dt.now().date())

In [9]:
#reference date for a static dataset, set the ref date to the maximum date in the dataset and add one
# adding one day can help calculate how many days have passed since the last transaction
reference_date = data['InvoiceDate'].max() + timedelta(days=1)
reference_date

Timestamp('2011-12-10 12:50:00')

In [24]:
#create the rfm model. group the data my CustomerID where all transactions
# for each CustomerID will be aggregated.
#For Recency Metric: lambda x receives the series of InvoiceDate and gets the difference bewtween
# last ref date and last date customer transacted. A lower number indicates a recent transaction
#For frequency metric: measures how often Customer makes a purchase, do this by counting
#For Monetary metric: amount of money each customer has made, sum up the TotalAmount
RFM=data.groupby('CustomerID').agg({
    'InvoiceDate': lambda x:(reference_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalAmount': 'sum'
})

In [25]:
#rename the columns to reflect RFM model
RFM.rename(columns={'InvoiceDate':'Recency','InvoiceNo':'Frequency','TotalAmount':'M_Value'}, inplace=True)
#RFM = RFM.sort_values(by='M_Value', ascending=False)
#I chose to remove the sorting so it doesn't introduce an element of unbiass towards the high spending customers
RFM.head()

,Recency,Frequency,M_Value
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [27]:
#Define quantiles
quantiles =RFM.quantile(q=[0.25,0.5,0.75])

#Assign RFM scores
def RScore(x,p,d):
    if x<= d[p][0.25]:
        return 4
    if x<= d[p][0.50]:
        return 3
    if x<= d[p][0.75]:
        return 2
    else:
        return 1
def FMScore(x, p, d):
    if x<= d[p][0.25]:
        return 1
    if x<= d[p][0.50]:
        return 2
    if x<= d[p][0.75]:
        return 3
    else:
        return 4
    
RFM['R']=RFM['Recency'].apply(RScore, args=('Recency',quantiles))
RFM['F']=RFM['Frequency'].apply(FMScore, args=('Frequency',quantiles))
RFM['M']=RFM['M_Value'].apply(FMScore, args=('M_Value',quantiles))

RFM.head()

#best customer is the one with a score of 4,4,4 hence can be target for marketing strategies

,Recency,Frequency,M_Value,R,F,M
CustomerID,,,,,,
12346.0,326,2,0.00,1,1,1
12347.0,2,182,4310.00,4,4,4
12348.0,75,31,1797.24,2,2,4
12349.0,19,73,1757.55,3,3,4
12350.0,310,17,334.40,1,1,2


In [31]:
#first we want to create a column RFM_Segment that displays the three values as a string
#get the sum of the the scores 
RFM['RFM_Segment']=RFM['R'].astype(str) + RFM['F'].astype(str) + RFM['M'].astype(str)
RFM['RFM_Score'] =RFM[['R','F', 'M']].sum(axis=1)
RFM.head()

,Recency,Frequency,M_Value,R,F,M,RFM_Segment,RFM_Score,RFM_Segment_Label
CustomerID,,,,,,,,,
12346.0,326,2,0.00,1,1,1,111,3,Bronze
12347.0,2,182,4310.00,4,4,4,444,12,Platinum
12348.0,75,31,1797.24,2,2,4,224,8,Silver
12349.0,19,73,1757.55,3,3,4,334,10,Platinum
12350.0,310,17,334.40,1,1,2,112,4,Bronze


In [32]:
#assign labels depending on the RFM_scores 
segment_labels =['Bronze', 'Silver','Platinum']
def assign_segment(score):
    if score <5:
        return 'Bronze'
    elif score <9:
        return 'Silver'
    else:
        return 'Platinum'

RFM['RFM_Segment_Label']= RFM['RFM_Score'].apply(assign_segment)
RFM.head()

,Recency,Frequency,M_Value,R,F,M,RFM_Segment,RFM_Score,RFM_Segment_Label
CustomerID,,,,,,,,,
12346.0,326,2,0.00,1,1,1,111,3,Bronze
12347.0,2,182,4310.00,4,4,4,444,12,Platinum
12348.0,75,31,1797.24,2,2,4,224,8,Silver
12349.0,19,73,1757.55,3,3,4,334,10,Platinum
12350.0,310,17,334.40,1,1,2,112,4,Bronze


In [33]:
#count how many customers are in each segment
segment_counts =RFM['RFM_Segment_Label'].value_counts().reset_index()
segment_counts.columns= ['RFM_Segment', 'Count']
segment_counts =segment_counts.sort_values('RFM_Segment')

#prints the number of count in each segment
segment_counts

,RFM_Segment,Count
2,Bronze,783
1,Platinum,1699
0,Silver,1890


In [34]:
#create the bar chart using Plotly
#each column to be given a different colour
fig =px.bar(segment_counts,
            x='RFM_Segment',
            y= 'Count',
            title ='Customer Distribution by RFM Segment',
            labels ={'RFM_Segment':'RFM Segment','Count':'Number of Customers'},
            color='RFM_Segment',
            #color_discrete_sequence=px.colors.qualitative.Pastel)
            color_discrete_sequence=px.colors.qualitative.Set2)

fig.show()

In [40]:
#create different customer segments based on their scores, RFM_Customer_Segments to be a place holder
#.loc accesses a group or rows and columns  by labels or a boolean array then assign the labels as per conditions set
# count the number of customers in each segment and assign to segment_counts, by counting occurences of unique Customer value in RFM_Customer_Segments

RFM['RFM_Customer_Segments']=''

RFM.loc[RFM['RFM_Score'] >=9, 'RFM_Customer_Segments'] ='Loyal'
RFM.loc[(RFM['RFM_Score'] >=6)& (RFM['RFM_Score'] <9), 'RFM_Customer_Segments'] ='Potential Loyal'
RFM.loc[(RFM['RFM_Score'] >=5)& (RFM['RFM_Score'] <6), 'RFM_Customer_Segments'] ='Dormant'
RFM.loc[(RFM['RFM_Score'] >=4)& (RFM['RFM_Score'] <5), 'RFM_Customer_Segments'] ='Critical'
RFM.loc[(RFM['RFM_Score'] >=3)& (RFM['RFM_Score'] <4), 'RFM_Customer_Segments'] ='Inactive'
segment_counts = RFM ['RFM_Customer_Segments'].value_counts().sort_index()
segment_counts

RFM_Customer_Segments
Critical            390
Dormant             515
Inactive            393
Loyal              1699
Potential Loyal    1375
Name: count, dtype: int64

In [41]:

segment_counts2 = RFM.groupby(['RFM_Score','RFM_Segment_Label','RFM_Customer_Segments']).size().reset_index(name='Count')

segment_counts2= segment_counts2.sort_values('Count', ascending=False)
segment_counts2

,RFM_Score,RFM_Segment_Label,RFM_Customer_Segments,Count
2,5,Silver,Dormant,515
9,12,Platinum,Loyal,470
3,6,Silver,Potential Loyal,469
5,8,Silver,Potential Loyal,467
7,10,Platinum,Loyal,442
4,7,Silver,Potential Loyal,439
6,9,Platinum,Loyal,413
0,3,Bronze,Inactive,393
1,4,Bronze,Critical,390
8,11,Platinum,Loyal,374


In [42]:
#Create a treemap
#the treemap will first be divided according to Segment_type i.e silver, platinum, bronze and each segment divided into loyal etc.
#size of each rectangle is proportional to the count
#
fig_treemap_segment_product=px.treemap(segment_counts2,
                                       path=['RFM_Segment_Label','RFM_Customer_Segments'],
                                       values ='Count',
                                       color= 'RFM_Segment_Label',
                                       color_discrete_sequence=px.colors.qualitative.Set2,
                                       title='RFM Customer Segments by Value')
                                       
#Display the treemap
fig_treemap_segment_product.show()

In [43]:
# Filter the DataFrame for 'Loyal' customers
Loyal_segment =RFM[RFM['RFM_Customer_Segments']=='Loyal']

In [46]:
fig=go.Figure()
# Add box plots for Recency, Frequency, and Monetary Value
fig.add_trace(go.Box(y=Loyal_segment['Recency'],name='Recency'))
fig.add_trace(go.Box(y=Loyal_segment['Frequency'],name='Frequency'))
fig.add_trace(go.Box(y=Loyal_segment['M_Value'],name='M_Value'))

#might want to remove the outliers in the data and continue with the analysis

In [49]:
#check correlation using a heatmap
Correlation_matrix =Loyal_segment[['R','F','M']].corr()

In [50]:
# Create the heatmap
fig_heatmap=go.Figure(data=go.Heatmap(
                z=Correlation_matrix.values,
                x=Correlation_matrix.columns,
                y=Correlation_matrix.columns,
                colorscale='Oranges',
                colorbar=dict(title='Correlation')
))
# Update layout with a title
fig_heatmap.update_layout(title='Correlation Matrix of RFM Values within Platinum Segment')

#Display the heatmap
fig_heatmap.show()

In [51]:


bar_colors=plotly.colors.qualitative.Set2

fig=go.Figure(data=[go.Bar(x=segment_counts.index,y=segment_counts.values,
                           marker=dict(color=bar_colors))])

Loyal_color='rgb(150,200,215)'

fig.update_traces(marker_color=[Loyal_color if segment =='Loyal'else bar_colors[i]
                                    for i, segment in enumerate(segment_counts.index)],
                marker_line_color='rgb(8,48,90)',
                marker_line_width=1,opacity=0.5)
                
#Update the layout
fig.update_layout(title='Comparison of RFM Segments',
                  xaxis_title='RFM Segments',
                  yaxis_title='Number of Customers',
                  showlegend=False)

#Display the figure
fig.show()

In [52]:
#custom shade of green
colors=['rgb(152, 251, 152)','rgb(50, 205, 50)','rgb(0, 100, 0)']

# Assuming segment_scores is your DataFrame
segment_scores=RFM.groupby('RFM_Customer_Segments')[['R','F','M']].mean().reset_index()

fig =go.Figure()

#Add bars for Recency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM_Customer_Segments'],
    y=segment_scores['R'],
    name='Recency Score',
    marker_color=colors[0]    
))

#Add bars for FRequency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM_Customer_Segments'],
    y=segment_scores['F'],
    name='Frequency Score',
    marker_color=colors[1]    
))

#Add bars for Monetary Value Score
fig.add_trace(go.Bar(
    x=segment_scores['RFM_Customer_Segments'],
    y=segment_scores['M'],
    name='M_Value Score',
    marker_color=colors[2]    
))
#Update the layout
fig.update_layout(
    title='Comparison of the RFM Segments based on Recency, Frequency and Monetary Value Scores',
    xaxis_title='RFM Segments',
    yaxis_title='Score',
    barmode='group',
    showlegend=True
)

#Display the grouped bar chart
fig.show()